In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)


In [4]:
train_df = pd.read_csv("https://storage.googleapis.com/uga-dsp/project2/files/X_small_train.csv")
train_df = train_df.rename(columns={'Sex (subj)': 'label'}) 
print(train_df)

     Unnamed: 0  Face ID                  Image File  Image Width  \
0        910982   961818    800402787_29b099b29f.jpg          500   
1        353124   374238   5700798023_d2d6e12786.jpg          375   
2        764002   807709   4123952950_e4c17543d4.jpg          333   
3        454440   482102  13869213505_c47163eddc.jpg          500   
4        477786   506985   5250685971_c99df536ef.jpg          375   
..          ...      ...                         ...          ...   
606      533736   566004   5077955544_6a68594f92.jpg          500   
607       41628    43718   4542784243_0d03b8faf5.jpg          500   
608      707689   748633   3894949684_0b1da6735a.jpg          500   
609      318698   337624   6033218437_9aa695d619.jpg          500   
610      539572   572111   8260180137_42348c4269.jpg          333   

     Image Height  X (top left)  Y (top left)  X (bottom right)  \
0             334           324           121               406   
1             500           122      

In [5]:
test_df = pd.read_csv("https://storage.googleapis.com/uga-dsp/project2/files/X_small_test.csv")
test_df = test_df.rename(columns={'Sex (subj)': 'label'}) 
print(test_df)

     Unnamed: 0  Face ID                 Image File  Image Width  \
0        447152   474293  5509578403_4ea41450b5.jpg          500   
1         99175   104646   908992675_3c5ad8477e.jpg          333   
2        767906   811780  5635971925_78be18d32c.jpg          500   
3         92665    97731  5055582974_b6302d7562.jpg          332   
4        770833   814834  7229890452_97905365f1.jpg          334   
..          ...      ...                        ...          ...   
258      398017   422032  4839519562_8b9332dfb8.jpg          500   
259      567886   601870  3988077711_81d28c60e1.jpg          500   
260      886885   936593  5340750171_3a063150cd.jpg          500   
261      584664   619447   134607743_91b7ba86f6.jpg          352   
262      364107   385979  4554837567_4cf952246d.jpg          500   

     Image Height  X (top left)  Y (top left)  X (bottom right)  \
0             375           182            96               255   
1             500           156           115    

In [6]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [7]:
spark_traindf = sqlContext.createDataFrame(train_df)
spark_testdf = sqlContext.createDataFrame(test_df)

In [28]:
feature_columns = spark_traindf.columns[3:-1]
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

In [29]:
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[assembler,lr])
p_model = p.fit(spark_traindf)



In [30]:
predictions = p_model.transform(spark_testdf)
predictions.select("Image File", "prediction").show(truncate=False)

+-------------------------+----------+
|Image File               |prediction|
+-------------------------+----------+
|5509578403_4ea41450b5.jpg|1.0       |
|908992675_3c5ad8477e.jpg |0.0       |
|5635971925_78be18d32c.jpg|1.0       |
|5055582974_b6302d7562.jpg|1.0       |
|7229890452_97905365f1.jpg|0.0       |
|440018518_9f1c6ec332.jpg |1.0       |
|5918397240_d1bd43433c.jpg|0.0       |
|250819946_dce648d292.jpg |1.0       |
|4798060474_0f88731eaf.jpg|1.0       |
|8973836058_86d5a7bfd4.jpg|1.0       |
|8608816325_927d5d5b8a.jpg|1.0       |
|1633590627_372446c3d1.jpg|0.0       |
|3368872317_88a021f9c5.jpg|1.0       |
|3430674374_9524bd80de.jpg|1.0       |
|6851820298_cec2b304dd.jpg|1.0       |
|8398646523_4709c60f99.jpg|0.0       |
|2948415544_698d034294.jpg|0.0       |
|5156347226_b9959f2186.jpg|0.0       |
|4852617666_3d6bcc3ddb.jpg|1.0       |
|3858652229_6d42808331.jpg|1.0       |
+-------------------------+----------+
only showing top 20 rows



In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
df = p_model.transform(spark_testdf)
df.show()

predictionAndLabels = df.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Training set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

+----------+-------+--------------------+-----------+------------+------------+------------+----------------+----------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+------------------+-----------------+--

Training set accuracy = 0.6958174904942965
